# Распределенные вычисления ДЗ-2 | Шамаев Онар Евгеньевич 

## Постановка задачи

Необходимо в потоковом формате считывать последнюю активность сообщества r/AskReddit платформы Reddit.
Целью будет найти самые встречающиеся слова.
Необходимо использовать Spark Streaming сохраняя данные на HDFS.

## План
1. Создание TCP сервера (RSS читателя заголовков)
2. Запуск HDFS
3. Написать MapReduce для Spark Streaming
4. Кеширование
5. Результаты

## 1. Создание TCP сервера (RSS читателя заголовков)

_Импорты библиотек._

In [ ]:
import feedparser
import socket
import time

Последние новости (в RSS формате) доступны по ссылке:
`https://www.reddit.com/r/AskReddit/new/.rss`

In [2]:
rss_link = 'https://www.reddit.com/r/AskReddit/new/.rss'

Определим программу читателя новостной ленты RSS и извлекающей оттуда заголовки, создающий TCP сокет и отправляющий их туда.

In [3]:
encoding = 'utf-8'


def rss2tcp_reader(ip, port, interval_sec: float = 30.0, ttl=None):
    last_stamp: time.struct_time | None = None
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((ip, port))
    s.listen(1)

    print(f'Awaiting connection at {ip}:{port}...')
    conn, addr = s.accept()
    print('Connected by ', addr)

    try:
        while True:
            if (ttl is not None) and last_stamp > ttl:
                print('Time to live passed. Exiting.')
                break

            feed = feedparser.parse(rss_link)
            if feed.status != 200:
                print(f'Bad response {feed.status} received! Exiting.')
                break

            _mx_stmp = None
            for i, entry in enumerate(feed.entries):
                title = entry.title
                time_stamp = entry.published_parsed

                if (last_stamp is None) or (last_stamp < time_stamp):
                    if _mx_stmp is None: _mx_stmp = time_stamp
                else:
                    break

                conn.send(title.encode(encoding) + b'\n')
            if _mx_stmp is not None:
                last_stamp = _mx_stmp

            time.sleep(interval_sec)
    except Exception as ex:
        print(f'Exception happened {ex}')
    finally:
        s.close()
        print(f'Closed connection')

Запустим программу в отдельном потоке. Сохраним поток в переменную `tcp_server`.

In [4]:
ip = 'localhost'
port = 9999
# tcp_server = multiprocessing.Process(target=rss2tcp_reader, args=(ip, port))
# tcp_server.start()

## 2. Запуск HDFS

Воспользуемся наработками 1 дз. Поднимем HDFS кластер из 1 namenode и 2 datanode.

UI кластера доступен по адресу `http://localhost:9870/`.

Сама система доступна по адресу `hdfs://localhost:8020`.

In [5]:
hdfs = 'hdfs://localhost:8020'

## 3. Написать MapReduce для Spark Streaming

_Импорты библиотек._

In [6]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import os

os.environ['PYSPARK_PYTHON'] = 'python'

Создадим Спарк-контекст и Стриминговый контекст.

In [7]:
from pyspark.sql import SparkSession

ctx = SparkContext(master="local[1]", appName="AskRedditWordCounter")
spark = SparkSession(ctx)

In [8]:
ctx_stream = StreamingContext(ctx, 1)

C:\Users\onaru\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\streaming\context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


PySpark UI доступен по адресу `http://localhost:4040`.

Будем использовать DStream, считывающий данные по TCP соединению нашего RSS читателя.

In [9]:
d_stream = ctx_stream.socketTextStream(ip, port)

Определим фильтр, для подготовки к разбиению заголовков на слова, и непосредственно разделитель на слова.

In [10]:
import string


def prepare(line: str) -> str:
    return line.translate({key: ' ' for key in string.punctuation})


def word_splitter(line: str) -> list[str]:
    return line.split(' ')

Определим обработчик RDD, получаемых

In [11]:
from pyspark import RDD


def rdd_handler(time_, rdd: RDD):
    print(f'RDD handled at {time_}')
    df = rdd.toDF(schema=["word", "count"])
    df.write.format('parquet').mode('overwrite').save(hdfs + '/askReddit.parquet')

Определим порядок действий джобы Spark Stream.

In [12]:
word_stream = d_stream.flatMap(lambda line: word_splitter(prepare(line.decode(encoding))))

words_mapped = word_stream.map(lambda word: (word, 1))

words_counts = words_mapped.reduceByKey(lambda x, y: x + y)

words_counts.foreachRDD(rdd_handler)

In [13]:
ctx_stream.start()
ctx_stream.awaitTermination(120)